<a href="https://colab.research.google.com/github/jh006/AARD-code/blob/main/cylindrical_metaatoms_cuda_v19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplo
import torch  # to use PyTorch (optimized tensor library for deep learning using GPU and CPU)
from torch import nn  #module torch. nn: diff classes, help  build nns
from torch.utils.data import DataLoader, Dataset
import numpy as np   # NumPy: Python library used for working w/arrays
                     #np.arrange, np.random.shuffle
import pandas as pd  #pandas: software library for data manipulation + analysis
                     #pd.read.csv   
​
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import torch.optim as optim
​
import torchvision
import math

In [ ]:

torch.cuda.is_available()
True
path = 'C:/Users/misha/Desktop/Python codes/Julia/training data/'
df1 = pd.read_csv(path + "parameters.csv", header=None)  #reads csv file; makes 0, 1, 2, 3 the heading instead of the first row of parameter values
df1.head() 
print(df1.head()) #gives first 5 rows
print(df1.shape[0]) #prints number of rows
       0     1     2     3
0  22.01  0.72  1.21  0.61
1  19.77  0.14  1.29  0.95
2  24.64  1.32  1.21  0.66
3  24.55  0.87  1.42  0.99
4  21.79  0.51  1.26  0.76
65611
# !gdown --id 1zGRT4aNV71ASxFXsG4MTZb0ZaqNslZSf #downloads real_t.csv file 
df2 = pd.read_csv(path + "real_t.csv", header=None)  #reads csv file; makes 0, 1, 2, 3 the heading instead of the first row of real_t values
df2.head() 
print(df2.head()) #prints first 5 rows
print(df2.shape[0]) #prints number of rows
        0         1         2         3         4         5         6    \
0 -0.923212 -0.913157 -0.901995 -0.889732 -0.876373 -0.861930 -0.846413   
1 -0.150980 -0.107935 -0.064665 -0.021280  0.022109  0.065395  0.108474   
2 -0.949474 -0.942679 -0.934842 -0.925965 -0.916052 -0.905110 -0.893145   
3 -0.015974  0.031830  0.077765  0.121477  0.162622  0.200877  0.235939   
4 -0.762562 -0.739172 -0.714570 -0.688787 -0.661854 -0.633810 -0.604694   

        7         8         9    ...       291       292       293       294  \
0 -0.829833 -0.812206 -0.793548  ...  0.245705  0.230979  0.215836  0.200349   
1  0.151243  0.193603  0.235461  ...  0.087291  0.242819  0.391866  0.527432   
2 -0.880165 -0.866178 -0.851196  ...  0.825729  0.855296  0.881989  0.905727   
3  0.267534  0.295418  0.319384  ...  0.435294  0.451729  0.466411  0.479366   
4 -0.574548 -0.543418 -0.511353  ... -0.068997 -0.047779 -0.019230  0.016327   

        295       296       297       298       299       300  
0  0.184594  0.168656  0.152626  0.136604  0.120697  0.105025  
1  0.643482  0.735414  0.800337  0.837132  0.846358  0.830031  
2  0.926430  0.944015  0.958400  0.969504  0.977244  0.981539  
3  0.490628  0.500236  0.508234  0.514671  0.519597  0.523061  
4  0.058388  0.106278  0.159172  0.216121  0.276085  0.337978  

[5 rows x 301 columns]
65611
#parameters
lr = 1e-6
# momentum = 0.9
dropout_rate = 0.3
td_ratio = 1-dropout_rate  # define percent of data that's training data
batch_size = 200
torch.manual_seed(0) # Sets the seed for generating random numbers
# will have the same random numbers each time, running the code on different computers -> same results
​
# split the full data into training and test
x_full=torch.tensor(df1.iloc[0:len(df1)].values).float().cuda()   #full x data from parameters.csv
y_full=torch.tensor(df2.iloc[0:len(df2)].values).float().cuda()   #full y data from real_t
​
inx=np.arange(len(df1)) # create array 0 to length(df1)-1, generates index
np.random.shuffle(inx)   #randomly shuffle the index of the data
 
dropout_rate = dropout_rate
td_ratio = 1-dropout_rate #percent training data
p= td_ratio  # define percent of data that's training data
train_size= math.ceil(int(td_ratio*(len(df1)))/batch_size)*batch_size # size of training data
# integer of (65611*0.7) divided by batch size (200) -> 230 (ceiling, round up)
# num of batch * 200 -> 46000 rows of data for training
# ^ prevent cases of decimal batches/batches of different sizes
training_idx, test_idx = inx[:train_size], inx[train_size:] # index of training and test data
x_train, x_test = x_full[training_idx,:], x_full[test_idx,:] #training x data and test data with shuffled inx 
y_train, y_test = y_full[training_idx,:], y_full[test_idx,:] #training y data and test data with shuffled inx 
​
#convert the data into N*1*4 shape or N*1*301 shape, need this format for calculating bilinear function
#[....       [[....]
# ....   ->  [....]
# ....]      [....]]
x_train=x_train.reshape(len(x_train),1,4)
x_test=x_test.reshape(len(x_test),1,4)
y_train=y_train.reshape(len(y_train),1,301)
y_test=y_test.reshape(len(y_test),1,301)
​
​
#print data shapes 
print('full x data shape:',x_full.shape, x_full.device)
print('x train data shape:', x_train.shape, x_train.device)
print('x test data shape:', x_test.shape, x_test.device)
print('y full data shape:',  y_full.shape, y_full.device)
print('y train data shape:', y_train.shape, y_train.device)
print('y test data shape:', y_test.shape, y_test.device)
full x data shape: torch.Size([65611, 4]) cuda:0  #parameters.csv data
x train data shape: torch.Size([46000, 1, 4]) cuda:0  
x test data shape: torch.Size([19611, 1, 4]) cuda:0
y full data shape: torch.Size([65611, 301]) cuda:0  #real data
y train data shape: torch.Size([46000, 1, 301]) cuda:0
y test data shape: torch.Size([19611, 1, 301]) cuda:0

# define dataset class
class SpectroData(Dataset):
  def __init__(self, data): #load data and convert to tensors
    self.data = data
  def __getitem__(self, index): #gives data from imported dataset
    return self.data[index] 
  def __len__(self):  #returns length of tensor
    return len(self.data)
#combine training data for dataloader
train_dataset1 = torch.utils.data.TensorDataset(x_train, y_train)  # all training data (for parameters.csv and real)
test_dataset1= torch.utils.data.TensorDataset(x_test, y_test)      # all test data ^
# ^ puts x_train and y_train batches next to each other so that rows​ line up, when taking data, get one row containing both param+real batches
#then after calculate loss + update x_train params, get y_pred and compare with corresponding y_train values 

train_dataset=SpectroData(train_dataset1)
test_dataset=SpectroData(test_dataset1)
​
#load x_train and y_train together
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False) 
# , num_workers=0, pin_memory=True) 
 
def f(x, W1,W2, V, b): #def bilinear function; x is a N*1*4
 Q=torch.randn(len(x),1,50).cuda() #Q is N*1*50  
 A = torch.randn(50,1).cuda() #define A
 F = torch.randn(50,1).cuda() #define F
 for i in range (0,len(x)): 
     e=torch.transpose(x[i],1,0).cuda() #transpose x from row to column (check) 
     et = torch.transpose(e,1,0).cuda()   #transpose e from row to column (check)
     esq = torch.mul(e, e).cuda()  #squares e
     for k in range (0, 50): 
         A[k, 0] = torch.mm(et,torch.mm(W1[k, :, :], e))  #puts values together -> 50x1 tensor
         F[k, 0] =torch.mm(et,torch.mm(W2[k, :, :], esq))  #matrix multiplication
     ecat = torch.cat((e, esq)) 
     L=torch.mm(V, ecat)
     Q[i]= torch.transpose(A + F + L + b, 1,0) #adds all four 50x1 tensors -> output
 return(Q)
 
class NeuralNetwork(nn.Module):   #https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.W1 = torch.nn.Parameter(torch.randn(50, 4, 4)) #define W, tensor, 50 layers of 4x4 matrices
        self.W2 = torch.nn.Parameter(torch.randn(50, 4, 4)) #define W, tensor, 50 layers of 4x4 matrices
        #W2 = torch.randn(50, 4, 4, requires_grad=True) #define W2
        self.V = torch.nn.Parameter(torch.randn(50,8))    #define V
        self.b = torch.nn.Parameter(torch.randn(50,1))    #define b        
        self.linear_relu_stack = nn.Sequential(   #nn.Sequential -> output of each layer as input of next layer
            nn.ReLU(),  #ReLU: activation function (defines output/what goes into next layer)
            nn.Linear(50, 500), # creates single layer feed forward network with 50 inputs and 500 outputs
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 301),
        )
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x): #def forward function
       Q=f(x, self.W1,self.W2,self.V, self.b)
       logits = self.linear_relu_stack(Q)
       return logits  
model = NeuralNetwork().cuda() 

from torch.autograd import Variable  # Variable wraps tensor, gives way to perform backpropagation
# tells function which variables/things to use
​
n_batches=math.ceil(len(x_train)/batch_size) # for printing the number of batches in train data
print("number of batches: ", n_batches)
​
optimizer = optim.Adam(model.parameters(), lr = lr) # see notes
​
loss_func = nn.MSELoss()
batch_loss = Variable(torch.FloatTensor(1)).zero_() 
# ^ only 1 value printed for loss -> (1), helps record + update gradient to get loss, start at 0
batch_loss.requires_grad = True
 
​# []: empty list
Trainloss = [] # loss of every epoch in iteration for training data
Testloss = [] # loss of every epoch in iteration for test data
r_squared = [] 
indices = []  # index of epochs
​
y_train1=y_train.reshape(len(y_train),301).cuda()  # reshaping real training data batches, [46000, 1, 301] -> [46000, 301], for calculating r-squared
y_train2 = y_train1.detach().cuda()  # returns new tensor, result doesn't require gradient because y_train is real data not params
y_pred = []  
​
num_epoch = 100
for epoch in range(num_epoch):
    i=0 #counter for batches
    for x_train_batch, y_train_batch in train_loader:  #all of this for one batch, repeats 230 times per epoch)
        x_train_batch, y_train_batch =(Variable(x_train_batch),Variable(y_train_batch))
        y_pred_batch = model(x_train_batch) #Forward pass: Compute predicted y by passing x through model
      #combine all y_pred_batch into y_pred
        if i==0: 
            y_pred =  y_pred_batch   #first batch: store y_pred_batch(results) in list y_pred
        else:
            y_pred = torch.cat((y_pred, y_pred_batch),0)  #second-230th batches: concatenate next result with list
         i=i+1
        batch_loss = loss_func(y_train_batch, y_pred_batch) #calculate MSE loss for each batch of predicted + real data
        if i%10==1: print("loss at batch",i , batch_loss) #every 10 batches, print a loss value (otherwise would print 23000 values)

        #update params     
        optimizer.zero_grad()
        batch_loss.backward(retain_graph=True)
        optimizer.step()
    loss_train = loss_func(y_pred, y_train.cuda()) #for each epoch, calculate loss (not for each batch)
    if epoch%5==1: print("epoch", epoch, "loss_train:", loss_train) #print loss every 5 epochs

#r-squared score
    y_pred1=y_pred.reshape(len(y_train),301)
    y_pred2 = y_pred1.detach()
    r2=r2_score(y_train2.cpu(), y_pred2.cpu())
    r_squared.append(r2)
    print("r2:", r_squared)
    
#compute and print loss
    Trainloss.append(loss_sum.item())
    print("trainloss:", Trainloss)
    indices.append(epoch+1)
​


torch.Size([46000, 1, 4])
number of batches:  230
loss at batch 1 tensor(147968.8750, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(142518.1094, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(157128.0469, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(140495.9531, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(132087.1719, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(124381.3672, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(122878.1406, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(121657.7578, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(126881.9219, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(112984.6328, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(104266.4766, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(99680.3516, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(109172.5938, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(106307.9609, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(93208.7656, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(91624.7891, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(92289.7109, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(93994.5312, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(92146.9297, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(89289.6641, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(85730.6172, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(87927.1406, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(81557.2344, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(73813.2188, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(71318.7422, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(78859.7656, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(70714., device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(66633.8438, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(62879.3125, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(62235.6055, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(61750.9805, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(64532.4258, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(57599.0664, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(53262.0039, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(51040.4766, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(56004.4023, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(54612.5898, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(47940.8555, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(47173.9531, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(47536.3984, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(48436.6680, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(47510.8008, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(46057.3203, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(44244.5430, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(45404.0195, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(42132.7930, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 1  loss_sum: tensor(56182.4414, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(38143.5469, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(36878.1992, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(40783.2734, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(36575.0312, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(34460.6562, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(32524.9844, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(32203.9473, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(31969.2539, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(33415.5781, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(29841.3828, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(27596.8398, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(26452.9434, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(29030.2324, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(28327.6738, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(24888.0215, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(24516.4863, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(24729.4414, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(25227.4590, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(24775.9766, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(24036.5156, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(23102.3379, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(23718.5137, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(22010.6035, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(19929.4297, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(19273.5078, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(21324.9258, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(19136.9609, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(18037.6953, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(17032.4277, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(16872.6250, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(16753.4395, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(17511.9062, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(15641.1250, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(14463.0039, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(13860.8955, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(15203.6455, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(14820.1377, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(13003.3682, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(12792.1523, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(12883.8125, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(13126.3262, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(12878.3789, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(12481.4736, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(11985.7324, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(12300.2354, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(11399.1738, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(10297.5908, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(9934.4795, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(10960.3672, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(9810.2764, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(9220.9092, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(8685.6963, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(8581.1943, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(8496.4717, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(8856.2979, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(7895.2441, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(7283.6313, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(6968.5825, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(7630.5342, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(7432.0771, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(6516.4946, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(6409.7412, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(6449.8774, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(6568.2749, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(6441.0010, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(6235.8281, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(5981.7578, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(6134.1279, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(5682.4722, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(5135.0122, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(4963.5342, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(5486.3193, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(4920.6431, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(4629.0649, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(4364.8657, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(4317.7056, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(4282.3135, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(4467.7808, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(3991.6760, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(3683.9817, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(3528.6023, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(3865.5125, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(3768.6477, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(3307.8293, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(3257.6211, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(3278.3220, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(3341.2495, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(3279.9968, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(3176.4104, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(3047.9995, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(3128.1104, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(2898.0288, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(2618.8713, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(2534.7209, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(2803.8147, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(2517.7280, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(2368.3242, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(2233.0300, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(2208.3274, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(2188.2690, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(2278.9119, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(2036.9395, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1877.9926, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1797.7971, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1967.4713, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1917.2373, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1682.5420, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1656.8536, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1664.7200, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1694.7695, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1664.1823, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1609.7275, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1542.9318, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1582.5690, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1462.9158, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 6  loss_sum: tensor(1980.8698, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1319.3126, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1276.2114, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1409.9813, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1265.7214, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1188.1200, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1118.2223, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1104.4832, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1093.7664, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1135.8926, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1016.0449, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(935.0669, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(893.7089, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(975.6324, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(949.3445, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(832.4737, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(819.3090, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(820.3124, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(832.6174, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(817.0916, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(788.0259, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(753.1632, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(771.6149, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(711.7666, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(641.5986, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(620.3168, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(684.8046, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(615.1443, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(575.8467, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(540.9599, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(533.3615, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(527.3300, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(545.1006, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(488.4258, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(449.0193, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(428.2271, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(466.1685, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(453.0579, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(397.1783, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(391.2365, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(389.8850, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(394.3483, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(387.9514, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(373.2677, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(355.5682, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(364.6151, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(335.3625, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(302.6497, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(292.4639, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(323.1316, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(291.3958, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(271.9118, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(255.1786, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(251.5240, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(248.2950, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(254.7869, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(229.6337, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(211.3905, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(201.1757, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(218.3692, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(212.3290, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(186.6153, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(184.6277, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(182.9351, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(184.0665, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(182.2634, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(175.0184, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(166.0256, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(171.0681, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(156.9755, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(142.5889, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(137.8197, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(153.0556, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(139.3111, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(129.6970, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(121.7241, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(120.4583, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(118.7689, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(120.5057, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(110.1791, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(102.1304, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(97.0745, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(105.0821, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(102.6577, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(90.9810, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(90.9595, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(89.5741, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(89.5549, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(89.9266, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(86.3848, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(81.6233, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(85.0884, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(78.1246, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(72.1936, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(69.9165, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(78.6587, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(72.9581, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(67.9630, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(64.0563, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(64.1399, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(63.3233, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(63.2683, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(59.3879, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(55.9598, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(53.2095, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(57.6309, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(56.8703, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(51.1988, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(52.1418, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(51.0628, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(50.6796, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(52.0518, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(50.1832, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(47.2819, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(50.2151, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(46.2991, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 11  loss_sum: tensor(58.3430, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(43.9612, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(42.6381, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(48.8549, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(46.3806, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(43.3649, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(41.0991, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(41.8182, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(41.3502, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(40.6982, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(39.3270, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(37.7557, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(35.9711, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(38.9943, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(38.9802, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(35.6673, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(36.9147, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(36.0657, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(35.6717, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(37.3163, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(36.1848, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(34.0417, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(36.7321, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(34.0530, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(33.1321, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(32.1123, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(37.3968, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(36.1317, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(33.9001, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(32.2216, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(33.3037, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(32.9592, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(32.1100, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(31.5175, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(30.6807, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(29.2190, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(31.7626, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(32.0311, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(29.5460, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(30.9140, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(30.2053, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(29.7703, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(31.4868, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(30.6201, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(28.7937, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(31.3652, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(29.1576, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(28.7798, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(27.7944, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(32.6851, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(31.8057, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(29.9107, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(28.4733, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(29.6573, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(29.3411, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(28.5015, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(28.2956, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(27.7687, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(26.4843, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(28.7800, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(29.1569, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(27.0132, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(28.3617, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(27.7577, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(27.3328, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(28.9670, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(28.2398, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(26.5494, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(29.0026, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(27.0123, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(26.8139, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(25.8526, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(30.5086, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(29.7750, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(28.0430, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(26.6913, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(27.9276, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(27.5624, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(26.7254, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(26.6538, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(26.1883, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(25.0245, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(27.1591, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(27.5235, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(25.5780, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(26.8787, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(26.3368, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(25.9405, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(27.4494, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(26.7794, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(25.1641, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(27.4925, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(25.6389, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(25.4950, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(24.5705, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(29.0115, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(28.2833, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(26.6544, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(25.3270, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(26.5744, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(26.1333, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(25.3164, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(25.3288, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(24.7914, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(23.7108, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(25.6718, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(26.0163, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(24.1729, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(25.3733, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(24.9020, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(24.5697, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(25.8838, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(25.2157, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(23.6267, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(25.8382, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(24.1393, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 16  loss_sum: tensor(25.4666, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(24.0037, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(23.1413, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(27.3905, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(26.6128, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(25.1458, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(23.8717, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(25.1463, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(24.6618, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(23.8903, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(24.0463, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(23.4727, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(22.5117, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(24.3157, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(24.6698, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(22.9364, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(24.0819, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(23.6846, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(23.4339, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(24.6073, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(23.9628, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(22.4464, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(24.5740, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(23.0071, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(22.8789, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(22.0914, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(26.1693, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(25.4078, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(24.0453, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(22.8367, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(24.0966, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(23.5906, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(22.8740, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(23.1079, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(22.4914, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(21.6257, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(23.3186, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(23.6656, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(22.0051, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(23.1119, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(22.7730, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(22.5727, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(23.6299, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(23.0176, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(21.5589, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(23.5995, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(22.1211, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(21.9898, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(21.2560, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(25.1852, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(24.4320, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(23.1486, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(21.9746, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(23.2178, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(22.7027, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(22.0101, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(22.2936, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(21.6447, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(20.8459, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(22.4409, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(22.7818, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(21.1860, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(22.2478, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(21.9520, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(21.7865, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(22.7457, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(22.1589, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(20.7435, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(22.6999, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(21.3030, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(21.1621, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(20.4744, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(24.2663, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(23.5145, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(22.3001, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(21.1540, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(22.3834, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(21.8554, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(21.1862, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(21.5159, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(20.8360, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(20.0934, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(21.5950, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(21.9301, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(20.3957, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(21.4143, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(21.1570, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(21.0172, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(21.8897, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(21.3233, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(19.9462, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(21.8233, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(20.5046, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(20.3572, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(19.7083, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(23.3667, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(22.6185, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(21.4657, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(20.3486, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(21.5647, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(21.0219, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(20.3766, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(20.7513, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(20.0430, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(19.3501, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(20.7626, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(21.0919, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(19.6165, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(20.5924, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(20.3744, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(20.2562, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(21.0444, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(20.4980, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(19.1587, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(20.9597, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(19.7143, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 21  loss_sum: tensor(20.6143, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(19.5607, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(18.9497, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(22.4767, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(21.7347, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(20.6360, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(19.5510, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(20.7547, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(20.1950, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(19.5713, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(19.9877, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(19.2541, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(18.6102, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(19.9342, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(20.2590, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(18.8433, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(19.7756, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(19.5938, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(19.4937, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(20.1989, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(19.6773, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(18.3751, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(20.1004, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(18.9272, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(18.7658, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(18.1928, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(21.5839, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(20.8492, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(19.7960, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(18.7488, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(19.9402, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(19.3612, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(18.7502, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(19.2021, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(18.4317, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(17.8407, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(19.0697, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(19.3767, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(17.9939, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(18.8784, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(18.7323, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(18.6598, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(19.2457, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(18.7435, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(17.4450, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(19.0995, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(17.9925, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(17.7669, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(17.2827, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(20.5061, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(19.7520, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(18.7892, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(17.7597, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(18.9283, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(18.3084, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(17.7445, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(18.2708, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(17.4349, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(16.9297, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(18.0393, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(18.3255, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(17.0261, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(17.8521, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(17.7748, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(17.7587, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(18.2343, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(17.7473, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(16.4843, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(18.0673, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(17.0718, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(16.8394, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(16.4020, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(19.4898, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(18.7289, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(17.8247, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(16.8541, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(18.0075, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(17.3858, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(16.8359, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(17.4292, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(16.5636, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(16.1237, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(17.1342, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(17.4324, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(16.2104, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(16.9776, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(16.9620, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(16.9593, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(17.3590, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(16.9013, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(15.6703, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(17.1857, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(16.2655, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(16.0421, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(15.6241, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(18.5977, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(17.8506, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(16.9953, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(16.0668, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(17.2052, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(16.5745, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(16.0480, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(16.6739, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(15.8022, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(15.4093, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(16.3364, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(16.6357, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(15.4818, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(16.2027, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(16.2304, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(16.2308, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(16.5712, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(16.1373, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(14.9344, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(16.3877, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(15.5310, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 26  loss_sum: tensor(16.2780, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(15.3073, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(14.9164, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(17.7723, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(17.0382, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(16.2254, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(15.3334, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(16.4556, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(15.8164, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(15.3112, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(15.9659, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(15.0830, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(14.7313, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(15.5781, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(15.8772, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(14.7791, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(15.4597, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(15.5232, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(15.5275, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(15.8121, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(15.4004, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(14.2243, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(15.6177, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(14.8192, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(14.5942, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(14.2267, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(16.9709, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(16.2507, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(15.4708, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(14.6191, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(15.7189, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(15.0755, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(14.5892, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(15.2713, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(14.3739, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(14.0631, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(14.8272, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(15.1334, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(14.0859, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(14.7273, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(14.8256, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(14.8328, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(15.0562, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(14.6719, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(13.5172, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(14.8545, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(14.1105, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(13.8885, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(13.5385, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(16.1720, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(15.4706, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(14.7241, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(13.9116, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(14.9935, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(14.3454, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(13.8770, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(14.5882, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(13.6784, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(13.4090, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(14.0961, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(14.4054, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(13.4092, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(14.0148, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(14.1439, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(14.1489, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(14.3262, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(13.9682, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(12.8357, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(14.1175, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(13.4273, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(13.2047, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(12.8773, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(15.4016, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(14.7146, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(14.0122, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(13.2305, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(14.2897, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(13.6399, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(13.1946, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(13.9211, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(13.0094, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(12.7789, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(13.3930, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(13.7021, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(12.7493, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(13.3254, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(13.4778, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(13.4856, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(13.6194, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(13.2862, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(12.1729, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(13.4068, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(12.7615, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(12.5433, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(12.2372, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(14.6519, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(13.9812, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(13.3209, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(12.5748, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(13.6048, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(12.9573, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(12.5339, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(13.2757, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(12.3625, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(12.1698, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(12.7123, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(13.0183, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(12.1142, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(12.6602, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(12.8349, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(12.8443, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(12.9411, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(12.6276, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(11.5392, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(12.7241, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(12.1231, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 31  loss_sum: tensor(12.7474, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(11.9127, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(11.6239, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(13.9333, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(13.2796, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(12.6570, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(11.9458, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(12.9466, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(12.3033, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(11.9003, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(12.6522, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(11.7465, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(11.5828, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(12.0601, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(12.3658, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(11.5053, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(12.0228, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(12.2157, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(12.2255, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(12.2891, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(11.9935, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(10.9339, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(12.0671, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(11.5089, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(11.3053, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(11.0329, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(13.2427, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(12.6055, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(12.0181, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(11.3390, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(12.3101, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(11.6731, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(11.2885, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(12.0476, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(11.1521, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(11.0152, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(11.4316, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(11.7357, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(10.9152, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(11.4067, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(11.6153, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(11.6261, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(11.6588, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(11.3798, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(10.3514, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(11.4326, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(10.9153, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(10.7194, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(10.4626, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(12.5756, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(11.9552, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(11.3996, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(10.7540, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(11.6942, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(11.0642, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(10.6976, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(11.4616, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(10.5786, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(10.4648, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(10.8252, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(11.1264, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(10.3436, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(10.8128, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(11.0347, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(11.0447, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(11.0493, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(10.7881, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(9.7892, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(10.8203, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(10.3423, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(10.1543, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(9.9116, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(11.9298, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(11.3266, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(10.8015, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(10.1902, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(11.0972, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(10.4744, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(10.1257, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(10.8909, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(10.0210, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(9.9279, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(10.2383, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(10.5370, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(9.7912, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(10.2393, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(10.4748, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(10.4795, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(10.4563, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(10.2115, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(9.2445, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(10.2222, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(9.7824, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(9.6041, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(9.3714, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(11.2847, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(10.6981, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(10.2131, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(9.6289, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(10.5110, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(9.9013, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(9.5638, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(10.3262, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(9.4614, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(9.3975, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(9.6609, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(9.9464, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(9.2578, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(9.6691, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(9.9301, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(9.9219, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(9.8760, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(9.6434, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(8.7234, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(9.6310, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(9.2286, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 36  loss_sum: tensor(9.7565, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(9.0704, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(8.8498, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(10.6669, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(10.0998, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(9.6510, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(9.0915, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(9.9470, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(9.3463, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(9.0300, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(9.7857, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(8.9297, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(8.8859, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(9.1092, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(9.3865, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(8.7298, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(9.1204, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(9.3888, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(9.3775, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(9.3150, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(9.0971, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(8.2065, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(9.0654, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(8.6926, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(8.5378, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(8.3288, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(10.0490, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(9.4834, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(9.0605, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(8.5226, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(9.3560, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(8.7457, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(8.4571, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(9.2221, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(8.3691, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(8.3438, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(8.5223, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(8.7908, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(8.1562, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(8.5436, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(8.8150, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(8.8212, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(8.7202, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(8.5112, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(7.6529, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(8.4722, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(8.1482, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(7.9900, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(7.8145, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(9.4559, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(8.9165, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(8.5292, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(8.0250, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(8.8302, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(8.2308, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(7.9650, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(8.7227, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(7.8880, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(7.8728, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(8.0215, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(8.2832, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(7.6784, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(8.0496, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(8.3229, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(8.3307, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(8.2125, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(8.0202, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(7.1948, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(7.9685, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(7.6784, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(7.5223, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(7.3657, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(8.9215, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(8.4024, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(8.0400, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(7.5615, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(8.3364, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(7.7524, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(7.5054, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(8.2515, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(7.4387, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(7.4323, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(7.5518, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(7.8063, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(7.2310, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(7.5881, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(7.8604, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(7.8689, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(7.7367, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(7.5586, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(6.7651, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(7.4946, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(7.2345, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(7.0824, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(6.9434, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(8.4160, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(7.9165, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(7.5792, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(7.1245, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(7.8689, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(7.3038, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(7.0718, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(7.8026, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(7.0131, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(7.0175, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(7.1088, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(7.3549, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(6.8085, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(7.1525, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(7.4214, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(7.4304, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(7.2878, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(7.1226, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(6.3595, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(7.0480, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(6.8143, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 41  loss_sum: tensor(7.2239, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(6.6659, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(6.5458, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(7.9358, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(7.4579, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(7.1439, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(6.7120, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(7.4254, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(6.8798, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(6.6609, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(7.3749, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(6.6104, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(6.6235, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(6.6907, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(6.9276, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(6.4081, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(6.7401, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(7.0038, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(7.0123, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(6.8632, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(6.7101, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(5.9758, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(6.6266, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(6.4160, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(6.2727, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(6.1692, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(7.4793, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(7.0254, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(6.7307, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(6.3213, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(7.0038, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(6.4786, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(6.2716, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(6.9665, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(6.2300, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(6.2492, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(6.2962, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(6.5231, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(6.0293, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(6.3493, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(6.6069, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(6.6143, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(6.4608, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(6.3192, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(5.6132, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(6.2279, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(6.0382, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(5.9002, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(5.8115, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(7.0454, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(6.6166, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(6.3383, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(5.9517, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(6.6034, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(6.0983, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(5.9036, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(6.5770, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(5.8687, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(5.8925, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(5.9227, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(6.1396, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(5.6690, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(5.9789, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(6.2296, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(6.2343, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(6.0801, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(5.9476, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(5.2704, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(5.8517, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(5.6789, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(5.5484, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(5.4711, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(6.6329, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(6.2277, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(5.9638, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(5.5999, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(6.2223, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(5.7371, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(5.5551, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(6.2061, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(5.5261, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(5.5529, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(5.5688, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(5.7748, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(5.3248, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(5.6250, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(5.8692, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(5.8713, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(5.7186, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(5.5945, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(4.9458, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(5.4957, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(5.3368, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(5.2143, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(5.1477, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(6.2381, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(5.8569, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(5.6071, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(5.2650, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(5.8585, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(5.3918, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(5.2231, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(5.8506, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(5.1988, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(5.2292, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(5.2335, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(5.4283, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(5.0002, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(5.2888, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(5.5266, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(5.5266, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(5.3758, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(5.2602, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(4.6390, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(5.1598, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(5.0162, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 46  loss_sum: tensor(5.3421, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(4.8997, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(4.8416, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(5.8654, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(5.5068, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(5.2716, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(4.9489, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(5.5139, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(5.0666, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(4.9101, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(5.5119, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(4.8909, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(4.9225, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(4.9185, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(5.1013, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(4.6948, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(4.9731, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(5.2020, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(5.2000, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(5.0520, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(4.9449, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(4.3518, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(4.8429, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(4.7131, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(4.6025, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(4.5520, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(5.5128, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(5.1759, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(4.9556, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(4.6493, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(5.1865, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(4.7585, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(4.6125, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(5.1876, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(4.5996, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(4.6304, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(4.6198, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(4.7911, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(4.4064, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(4.6748, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(4.8931, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(4.8888, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(4.7447, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(4.6466, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(4.0807, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(4.5423, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(4.4260, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(4.3195, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(4.2763, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(5.1773, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(4.8614, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(4.6559, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(4.3654, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(4.8731, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(4.4673, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(4.3295, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(4.8799, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(4.3231, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(4.3528, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(4.3364, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(4.4973, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(4.1329, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(4.3925, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(4.5989, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(4.5923, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(4.4542, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(4.3648, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(3.8243, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(4.2576, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(4.1535, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(4.0512, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(4.0160, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(4.8584, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(4.5644, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(4.3714, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(4.0981, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(4.5755, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(4.1925, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(4.0632, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(4.5872, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(4.0614, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(4.0904, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(4.0690, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(4.2214, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(3.8754, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(4.1269, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(4.3202, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(4.3116, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(4.1793, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(4.0987, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(3.5831, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(3.9908, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(3.8961, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(3.7994, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(3.7698, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(4.5572, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(4.2836, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(4.1034, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(3.8449, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(4.2946, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(3.9329, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(3.8121, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(4.3098, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(3.8142, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(3.8418, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(3.8162, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(3.9610, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(3.6320, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(3.8759, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(4.0557, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(4.0452, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(3.9174, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(3.8458, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(3.3535, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(3.7389, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(3.6524, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 51  loss_sum: tensor(3.9038, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(3.5600, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(3.5361, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(4.2702, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(4.0152, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(3.8446, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(3.6022, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(4.0266, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(3.6860, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(3.5737, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(4.0430, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(3.5792, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(3.6037, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(3.5757, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(3.7140, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(3.4009, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(3.6350, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(3.8041, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(3.7923, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(3.6700, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(3.6083, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(3.1387, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(3.5020, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(3.4229, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(3.3368, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(3.3170, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(4.0013, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(3.7643, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(3.6061, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(3.3763, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(3.7766, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(3.4543, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(3.3507, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(3.7944, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(3.3581, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(3.3812, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(3.3516, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(3.4813, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(3.1846, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(3.4080, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(3.5682, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(3.5549, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(3.4382, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(3.3843, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(2.9369, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(3.2791, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(3.2062, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(3.1259, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(3.1106, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(3.7477, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(3.5275, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(3.3805, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(3.1630, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(3.5406, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(3.2359, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(3.1407, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(3.5589, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(3.1487, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(3.1712, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(3.1403, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(3.2613, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(2.9806, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(3.1927, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(3.3445, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(3.3310, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(3.2192, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(3.1720, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(2.7470, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(3.0679, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(3.0012, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(2.9273, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(2.9161, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(3.5075, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(3.3034, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(3.1670, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(2.9622, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(3.3176, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(3.0302, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(2.9422, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(3.3359, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(2.9501, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(2.9724, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(2.9408, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(3.0533, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(2.7891, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(2.9886, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(3.1323, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(3.1197, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(3.0128, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(2.9703, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(2.5691, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(2.8676, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(2.8059, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(2.7392, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(2.7333, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(3.2789, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(3.0904, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(2.9644, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(2.7716, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(3.1053, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(2.8372, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(2.7548, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(3.1230, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(2.7611, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(2.7831, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(2.7507, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(2.8544, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(2.6092, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(2.7942, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(2.9303, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(2.9170, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(2.8159, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(2.7795, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(2.4010, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(2.6774, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(2.6212, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 56  loss_sum: tensor(2.8157, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(2.5596, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(2.5574, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(3.0613, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(2.8878, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(2.7700, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(2.5912, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(2.9051, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(2.6523, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(2.5769, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(2.9221, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(2.5819, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(2.6025, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(2.5701, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(2.6676, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(2.4382, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(2.6117, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(2.7391, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(2.7262, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(2.6306, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(2.5989, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(2.2423, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(2.4991, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(2.4475, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(2.3908, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(2.3928, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(2.8583, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(2.6975, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(2.5894, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(2.4217, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(2.7170, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(2.4790, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(2.4095, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(2.7342, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(2.4149, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(2.4350, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(2.4010, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(2.4936, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(2.2772, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(2.4414, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(2.5595, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(2.5471, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(2.4578, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(2.4299, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(2.0946, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(2.3336, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(2.2856, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(2.2341, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(2.2383, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(2.6683, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(2.5202, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(2.4201, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(2.2638, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(2.5404, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(2.3172, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(2.2527, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(2.5570, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(2.2588, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(2.2782, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(2.2425, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(2.3301, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(2.1258, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(2.2824, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(2.3911, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(2.3792, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(2.2958, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(2.2722, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.9570, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(2.1796, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(2.1351, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(2.0878, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(2.0932, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(2.4897, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(2.3545, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(2.2619, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(2.1160, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(2.3747, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(2.1657, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(2.1057, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(2.3900, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(2.1125, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(2.1309, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(2.0943, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(2.1770, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.9838, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(2.1337, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(2.2334, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(2.2221, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(2.1435, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(2.1247, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.8283, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(2.0358, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.9945, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.9509, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.9568, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(2.3225, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(2.1997, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(2.1140, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.9774, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(2.2193, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(2.0236, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.9677, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(2.2328, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.9753, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.9925, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.9560, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(2.0338, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.8511, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.9945, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(2.0857, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(2.0753, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(2.0010, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.9865, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.7082, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.9014, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.8629, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 61  loss_sum: tensor(2.0075, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.8230, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.8293, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(2.1657, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(2.0550, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.9756, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.8476, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(2.0736, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.8905, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.8385, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(2.0853, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.8468, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.8627, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.8273, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.8998, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.7272, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.8638, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.9475, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.9379, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.8678, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.8566, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.5959, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.7758, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.7394, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.7037, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.7104, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(2.0185, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.9200, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.8466, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.7257, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.9371, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.7660, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.7175, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.9471, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.7265, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.7407, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.7069, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.7746, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.6116, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.7411, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.8180, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.8098, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.7432, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.7349, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.4913, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.6587, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.6242, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.5925, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.5993, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.8807, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.7939, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.7260, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.6119, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.8092, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.6496, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.6046, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.8179, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.6145, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.6273, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.5946, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.6578, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.5041, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.6261, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.6965, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.6910, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.6268, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.6211, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.3939, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.5492, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.5169, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.4887, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.4950, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.7522, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.6767, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.6133, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.5055, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.6903, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.5416, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.4997, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.6974, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.5104, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.5216, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.4897, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.5484, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.4041, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.5188, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.5832, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.5805, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.5189, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.5150, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.3035, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.4471, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.4162, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.3920, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.3976, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.6330, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.5678, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.5080, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.4063, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.5797, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.4413, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.4019, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.5847, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.4133, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.4234, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.3917, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.4460, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.3112, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.4191, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.4779, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.4777, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.4185, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.4157, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.2189, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.3519, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.3221, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 66  loss_sum: tensor(1.4292, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.3018, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.3070, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.5223, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.4664, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.4098, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.3139, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.4764, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.3483, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.3109, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.4797, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.3228, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.3325, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.3003, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.3505, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.2246, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.3261, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.3800, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.3810, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.3248, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.3232, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.1398, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.2631, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.2345, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.2176, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.2223, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.4192, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.3714, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.3183, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.2281, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.3801, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.2613, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.2258, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.3820, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.2380, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.2476, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.2156, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.2620, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.1438, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.2395, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.2888, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.2911, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.2374, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.2373, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(1.0658, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.1806, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.1531, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.1391, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.1435, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.3236, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.2825, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.2335, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.1484, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.2904, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.1803, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.1466, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.2909, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.1590, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.1682, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.1372, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.1798, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(1.0688, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.1591, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.2035, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.2072, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.1563, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.1577, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.9969, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.1041, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.0779, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(1.0660, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.0703, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.2347, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.1994, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.1547, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.0741, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.2066, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.1048, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.0727, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.2057, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.0855, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.0940, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(1.0645, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.1033, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.9985, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.0841, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.1242, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.1289, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.0807, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.0834, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.9326, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(1.0331, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(1.0081, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.9980, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(1.0025, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.1520, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.1217, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.0808, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(1.0049, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.1285, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(1.0340, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(1.0039, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.1264, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(1.0169, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(1.0251, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.9967, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(1.0321, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.9329, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(1.0141, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(1.0509, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(1.0562, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(1.0102, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(1.0141, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.8730, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.9672, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.9432, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 71  loss_sum: tensor(1.0222, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.9347, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.9399, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.0756, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(1.0493, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(1.0119, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.9406, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(1.0557, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.9681, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.9399, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(1.0525, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.9533, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.9612, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.9332, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.9657, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.8723, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.9495, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.9833, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.9887, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.9449, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.9500, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.8178, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.9061, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.8834, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.8760, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.8821, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(1.0048, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.9823, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.9478, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.8812, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.9881, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.9071, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.8807, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.9843, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.8945, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.9018, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.8745, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.9038, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.8161, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.8894, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.9203, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.9260, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.8847, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.8908, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.7667, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.8495, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.8281, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.8216, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.8284, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.9395, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.9203, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.8887, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.8260, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.9254, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.8504, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.8260, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.9213, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.8400, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.8465, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.8200, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.8467, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.7645, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.8342, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.8620, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.8677, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.8294, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.8358, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.7197, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.7970, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.7768, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.7714, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.7786, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.8792, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.8628, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.8343, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.7750, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.8674, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.7983, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.7755, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.8631, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.7894, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.7953, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.7695, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.7941, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.7168, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.7832, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.8077, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.8137, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.7783, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.7848, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.6762, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.7481, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.7294, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.7252, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.7326, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.8235, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.8096, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.7836, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.7278, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.8136, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.7503, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.7287, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.8092, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.7423, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.7478, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.7228, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.7454, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.6726, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.7359, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.7574, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.7641, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.7308, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.7376, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.6360, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.7027, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.6855, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 76  loss_sum: tensor(0.7408, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.6824, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.6900, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.7723, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.7607, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.7372, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.6843, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.7635, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.7062, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.6851, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.7588, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.6986, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.7036, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.6793, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.7004, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.6319, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.6916, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.7107, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.7180, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.6863, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.6937, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.5987, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.6602, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.6446, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.6421, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.6500, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.7246, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.7154, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.6941, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.6443, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.7174, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.6651, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.6449, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.7124, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.6582, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.6626, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.6392, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.6587, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.5946, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.6508, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.6681, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.6756, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.6461, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.6536, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.5647, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.6217, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.6070, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.6055, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.6136, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.6811, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.6738, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.6543, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.6077, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.6752, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.6271, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.6081, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.6693, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.6212, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.6249, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.6025, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.6207, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.5605, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.6136, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.6290, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.6367, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.6092, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.6167, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.5337, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.5864, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.5720, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.5718, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.5801, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.6410, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.6353, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.6175, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.5737, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.6362, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.5921, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.5744, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.6298, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.5867, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.5901, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.5687, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.5856, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.5291, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.5795, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.5932, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.6007, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.5752, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.5825, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.5051, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.5539, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.5398, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.5407, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.5493, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.6040, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.5995, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.5836, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.5422, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.6003, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.5597, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.5434, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.5935, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.5553, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.5581, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.5376, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.5531, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.5003, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.5481, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.5601, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.5678, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.5437, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.5508, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.4785, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.5243, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.5103, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 81  loss_sum: tensor(0.5509, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.5121, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.5209, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.5699, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.5666, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.5526, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.5133, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.5672, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.5299, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.5147, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.5603, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.5262, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.5287, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.5087, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.5231, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.4736, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.5192, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.5296, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.5374, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.5147, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.5216, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.4540, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.4969, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.4832, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.4858, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.4947, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.5386, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.5361, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.5239, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.4866, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.5367, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.5026, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.4884, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.5297, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.4994, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.5017, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.4820, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.4954, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.4490, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.4927, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.5013, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.5095, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.4880, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.4947, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.4310, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.4717, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.4584, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.4616, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.4705, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.5097, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.5080, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.4976, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.4619, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.5086, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.4775, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.4639, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.5016, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.4747, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.4769, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.4576, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.4698, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.4262, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.4683, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.4749, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.4836, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.4633, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.4699, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.4098, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.4484, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.4353, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.4392, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.4482, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.4830, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.4822, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.4733, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.4392, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.4826, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.4545, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.4413, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.4755, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.4520, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.4540, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.4351, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.4462, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.4052, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.4455, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.4505, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.4595, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.4403, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.4471, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.3903, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.4270, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.4140, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.4186, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.4277, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.4586, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.4584, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.4511, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.4182, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.4586, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.4332, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.4205, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.4516, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.4309, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.4329, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.4144, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.4245, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.3858, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.4246, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.4280, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.4372, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.4191, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.4262, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.3725, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.4072, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.3945, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 86  loss_sum: tensor(0.4248, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.3995, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.4087, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.4360, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.4366, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.4305, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3989, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.4363, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.4136, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.4012, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.4295, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.4115, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.4134, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.3953, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.4044, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.3680, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.4054, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.4072, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.4166, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.3994, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.4068, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.3560, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.3889, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.3767, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.3818, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.3911, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.4151, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.4165, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.4113, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3812, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.4157, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3955, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.3835, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.4093, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3936, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3955, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.3777, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.3858, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.3517, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.3878, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.3882, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3976, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.3812, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.3891, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.3410, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.3721, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.3602, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.3656, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.3748, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.3958, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3980, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3936, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3648, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3967, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3788, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.3671, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.3906, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3770, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3790, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.3616, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.3687, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.3367, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.3713, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.3706, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3802, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.3645, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.3727, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.3271, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.3567, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.3451, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.3505, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.3598, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.3781, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3809, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3771, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3497, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3792, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3634, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.3520, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.3735, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3617, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3637, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.3467, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.3531, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.3229, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.3561, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.3545, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3643, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.3491, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.3576, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.3143, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.3424, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.3313, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.3364, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.3459, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.3616, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3650, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3619, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3357, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3631, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3491, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.3379, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.3577, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3476, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3495, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.3331, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.3386, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.3102, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.3421, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.3396, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3497, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.3349, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.3435, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.3025, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.3292, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.3184, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 91  loss_sum: tensor(0.3407, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.3234, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.3330, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.3464, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3504, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3478, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3229, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3484, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3359, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.3249, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.3430, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3346, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3365, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.3206, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.3253, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2984, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.3291, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.3260, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3361, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.3217, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.3305, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.2915, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.3170, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.3065, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.3112, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.3211, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.3326, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3369, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3346, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3111, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3348, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3236, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.3129, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.3296, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3226, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3245, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.3091, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.3130, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2876, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.3171, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.3135, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3237, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.3095, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.3186, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.2814, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.3056, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.2953, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.2998, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.3100, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.3198, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3244, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3225, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.3001, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3223, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3123, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.3018, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.3171, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3115, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3134, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.2986, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.3017, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2775, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.3061, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.3019, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3122, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.2983, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.3075, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.2720, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.2952, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.2850, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.2894, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.2999, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.3081, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3129, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3113, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.2900, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3107, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.3018, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.2915, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.3056, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.3011, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.3031, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.2888, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.2913, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2682, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.2959, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.2912, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.3015, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.2879, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.2972, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.2634, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.2855, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.2755, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.2797, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.2905, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.2973, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.3023, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.3010, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.2806, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.3001, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.2920, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.2821, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.2949, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.2915, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.2936, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.2798, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.2817, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2596, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.2866, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.2814, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.2917, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.2783, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.2878, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.2554, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.2765, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.2666, device='cuda:0', grad_fn=<MseLossBackward0>)
epoch 96  loss_sum: tensor(0.2842, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.2708, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.2819, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.2873, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.2924, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.2914, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.2719, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.2901, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.2830, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.2733, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.2851, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.2826, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.2847, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.2715, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.2728, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2518, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.2779, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.2723, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.2826, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.2694, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.2789, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.2479, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.2682, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.2585, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.2626, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.2738, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.2781, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.2833, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.2825, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.2639, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.2809, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.2746, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.2652, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.2760, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.2742, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.2765, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.2638, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.2645, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2445, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.2698, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 161 tensor(0.2638, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 171 tensor(0.2742, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 181 tensor(0.2612, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 191 tensor(0.2708, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 201 tensor(0.2410, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 211 tensor(0.2605, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 221 tensor(0.2510, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 1 tensor(0.2550, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 11 tensor(0.2664, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 21 tensor(0.2696, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 31 tensor(0.2748, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 41 tensor(0.2743, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 51 tensor(0.2564, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 61 tensor(0.2724, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 71 tensor(0.2668, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 81 tensor(0.2576, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 91 tensor(0.2675, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 101 tensor(0.2665, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 111 tensor(0.2688, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 121 tensor(0.2566, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 131 tensor(0.2567, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 141 tensor(0.2377, device='cuda:0', grad_fn=<MseLossBackward0>)
loss at batch 151 tensor(0.2623, device='cuda:0', grad_fn=<MseLossBackward0>)


IndentationError: ignored